In [1]:
import numpy as np
import pandas as pd
from reformat import *
from architecture.WordSegPreProcessing import *

fn = "train.tsv"
x, y = file_to_table(read_file(fn))


## Prepping Data

In [2]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.1)
preprocessor = WordSegPreProcessing(X_train, y_train, segment_to_tag)

In [3]:
X_train, y_train = preprocessor.x, preprocessor.y
X_test, y_test = preprocessor.extract_pairs(X_test, y_test)


## Supervised NGramTagger with Backoff

In [4]:
from architecture.NgramSupervisedTagger import NGramSupervisedTagger
ng = NGramSupervisedTagger(X_train, y_train, ngram_choice=2)
ng.create_n_gram_tagger()
ng.f1_by_tags(X_test, y_test)["I"] #F1 score for just I tags

/Users/a8nguyen/opt/anaconda3/lib/python3.7/site-packages/sklearn/feature_extraction/image.py:167: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  dtype=np.int):
/Users/a8nguyen/opt/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/least_angle.py:30: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details

0.8827667057444314

## Supervised HMM

In [5]:
from architecture.HMMSupervisedTagger import HMMSupervisedTagger
hmm = HMMSupervisedTagger(X_train, y_train)
hmm.train()
hmm.f1_by_tags(X_test, y_test)["I"]


0.8792497069167644

The same thing as HMM Tagger but instead of just using the provided characters, I manually engineered some features (probably not the best bleh)... You can check it out in `architecture.WordSegPreProcessing.apply_features`

In [6]:
from architecture.HMMSupervisedTagger import HMMSupervisedTagger
hmm2 = HMMSupervisedTagger(preprocessor.generate_features(X_train), y_train)
hmm2.train()
hmm2.f1_by_tags(preprocessor.generate_features(X_test), y_test)["I"]


0.9383177570093459

In [7]:
hmm2.tagger.best_path_simple(preprocessor.apply_features(X_test[3]))

['B', 'B', 'B', 'B', 'I', 'B', 'B']

## Unsupervised

In [8]:
from architecture.utils import *
x, y = file_to_table(read_file(fn))
feedX, feedY = preprocessor.extract_pairs(x[:650], y[:650])


In [9]:
testX2, testY2 = preprocessor.extract_pairs(x[650:], y[650:])


In [10]:
feedX = list(map(lambda x: preprocessor.let2index(x), feedX ))
feedY = list(map(lambda y: preprocessor.tag2index(y), feedY))
testX2 = list(map(lambda x: preprocessor.let2index(x), testX2))
testY2 = list(map(lambda x: preprocessor.tag2index(x), testY2))


In [11]:
#initialize with some probs by running HMM on the feeder set

import nltk
import numpy as np
init_tmat = np.zeros((len(preprocessor.index_tag),
                     len(preprocessor.index_tag)))

init_emission = np.zeros(
    (len(preprocessor.index_tag), len(preprocessor.index_vocab)))
trainer = nltk.HiddenMarkovModelTrainer()
tagger = trainer.train_supervised(
           tuple_xy4nltk(feedX, feedY))
for k in tagger._transitions.keys():
    for v in tagger._transitions[k].samples():
        init_tmat[k][v] = tagger._transitions[k].prob(v)
    for let in tagger._outputs[k].samples():
        init_emission[k][let] = tagger._outputs[k].prob(let)
init_state_distrib = np.array([0.5, 0.5])  # always start w B

#init_tmat


In [12]:
#add 0.05 alpha smoothing here cause I can't absorb to itself but heurestically, we know it is not true
init_tmat[preprocessor.index_tag["I"]][preprocessor.index_tag["I"]] += 0.05
init_tmat[preprocessor.index_tag["I"]][preprocessor.index_tag["B"]] -= 0.05
print(init_tmat)


[[0.89451595 0.10548405]
 [0.95       0.05      ]]


In [13]:
from architecture.Unsupervised import UnSupervised
unsupervised = UnSupervised(init_tmat, init_emission, init_state_distrib)


In [14]:
y_predicted = []
for ind in range(len(testX2)):
    tmat, emission, start, end = unsupervised.baum_welch(testX2[ind], 10)
    resultant = unsupervised.viterbi(testX2[ind], emission, tmat, {i: v for i, v in enumerate(start)})
    resultant[0] = preprocessor.index_tag["B"]
    y_predicted.append(resultant)


/Users/a8nguyen/Desktop/undergrad/fall22/csci4845/kreyol_segmentation/Krey-l-Segmentation/architecture/Unsupervised.py:105: RuntimeWarning: invalid value encountered in double_scalars
  new_tmat[sj][si] = transition_aux[si][sj]/norm_factor
/Users/a8nguyen/Desktop/undergrad/fall22/csci4845/kreyol_segmentation/Krey-l-Segmentation/architecture/Unsupervised.py:108: RuntimeWarning: invalid value encountered in double_scalars
  if np.isnan(emission_aux[si][o]/norm_factor):
/Users/a8nguyen/Desktop/undergrad/fall22/csci4845/kreyol_segmentation/Krey-l-Segmentation/architecture/Unsupervised.py:116: RuntimeWarning: invalid value encountered in double_scalars
  for si, term in enumerate(prod_alpha_beta_normed)]
/Users/a8nguyen/Desktop/undergrad/fall22/csci4845/kreyol_segmentation/Krey-l-Segmentation/architecture/Unsupervised.py:71: RuntimeWarning: invalid value encountered in true_divide
  sum(prod_alpha_beta)  # normalized alpha dot beta
/Users/a8nguyen/Desktop/undergrad/fall22/csci4845/kreyol_se

In [15]:
f1_by_tags(y_predicted, testY2)[preprocessor.index_tag["I"]]


0.7553418803418803

Running on test

supervision test

In [16]:
test_file = "test.txt"

In [17]:
xTest = []
for line in read_file(test_file):
    xTest.append(line.strip())

In [18]:
xTest_reformatted, _ = preprocessor.extract_pairs(xTest, xTest)

In [19]:
yPred_super = list(map(lambda i: hmm2.tagger.best_path_simple(preprocessor.apply_features(i)), xTest_reformatted))

In [20]:
#super_spell = open('super_spell.txt', 'w')
#for i in range(len(yPred_super)):
#    print(tag_to_segment(
#    yPred_super[i], xTest[i]),
#    file=super_spell)


semi supervision test

In [21]:
yPred_semi = []
for ind in range(len(xTest_reformatted)):
    curWord = preprocessor.let2index(xTest_reformatted[ind])
    tmat, emission, start, end = unsupervised.baum_welch(curWord, 10)
    resultant = unsupervised.viterbi(curWord, emission, tmat, {
                                     i: v for i, v in enumerate(start)})
    resultant[0] = preprocessor.index_tag["B"]
    resultant_to_BI =  list(map(lambda y: preprocessor.index2tag[y], resultant))
    yPred_semi.append(resultant_to_BI)


In [24]:
yPred_semi[2]

['B', 'B', 'B', 'B', 'B']

In [25]:
#semi_spell = open('semi_spell.txt', 'w')
#for i in range(len(yPred_semi)):
#    print(tag_to_segment(
#        yPred_semi[i], xTest[i]),
#        file=semi_spell)
